In [1]:
import requests as req


# Desafio 1

Faça uma requisição GET à API do Github para obter as informações do perfil de um usuário específico. Para isso, utilize a biblioteca Requests do Python e a URL https://api.github.com/users/{username}, substituindo {username} pelo nome de usuário desejado.

Com o objeto response retornado pela requisição, imprima na tela as seguintes informações sobre a requisição:

* status code da requisição, para analisar se foi bem sucedida ou não;
* conteúdo da requisição, para conferir se os dados de usuário vieram corretamente;
* URL da requisição feita, para garantir que a url utilizada na requisição foi a correta.

Após conferir isso, analise novamente o conteúdo retornado pela requisição e imprima na tela apenas as seguintes informações sobre a pessoa usuária em questão:

* Nome
* Nome de usuário
* Número de repositórios públicos
* Data de criação da conta no GitHub


In [2]:
username = 'not-valid-user'

print('Abaixo veremos uma request que não vai dar certo por conta do user não existir.')

req.get(f'https://api.github.com/users/{username}').json()


Abaixo veremos uma request que não vai dar certo por conta do user não existir.


{'message': 'Not Found',
 'documentation_url': 'https://docs.github.com/rest',
 'status': '404'}

In [3]:
username = 'juanveronez'
res = req.get(f'https://api.github.com/users/{username}')


In [4]:
print(f'Status Code: {res.status_code}')
print(f'Url: {res.url}')
print(f'\nConteúdo da requisição:\n{res.json()}')


Status Code: 200
Url: https://api.github.com/users/juanveronez

Conteúdo da requisição:
{'login': 'juanveronez', 'id': 37226530, 'node_id': 'MDQ6VXNlcjM3MjI2NTMw', 'avatar_url': 'https://avatars.githubusercontent.com/u/37226530?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/juanveronez', 'html_url': 'https://github.com/juanveronez', 'followers_url': 'https://api.github.com/users/juanveronez/followers', 'following_url': 'https://api.github.com/users/juanveronez/following{/other_user}', 'gists_url': 'https://api.github.com/users/juanveronez/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/juanveronez/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/juanveronez/subscriptions', 'organizations_url': 'https://api.github.com/users/juanveronez/orgs', 'repos_url': 'https://api.github.com/users/juanveronez/repos', 'events_url': 'https://api.github.com/users/juanveronez/events{/privacy}', 'received_events_url': 'https://api.github.com/users/

In [5]:
json = res.json()
type(json)


dict

In [6]:
print(f'Nome: {json['name']}')
print(f'Nome de usuário: {json['login']}')
print(f'Número de repositórios públicos: {json['public_repos']}')
print(f'Data de criação da conta no GitHub: {json['created_at']}')


Nome: Juan Veronez
Nome de usuário: juanveronez
Número de repositórios públicos: 8
Data de criação da conta no GitHub: 2018-03-09T20:36:08Z


In [7]:
from os import getenv
from dotenv import load_dotenv

load_dotenv()

access_token = getenv('GITHUB_ACCESS_TOKEN')

headers = {'X-GitHub-Api-Version': '2022-11-28',
           'Authorization': f'Bearer {access_token}'}


In [8]:
username = 'amzn'
url = f"https://api.github.com/users/{username}/followers"

response = req.get(url, headers)
followers = response.json()


In [14]:
followers_lists = []
while True:
    try:
        list_index = len(followers_lists) + 1
        page_url = f'{url}?per_page=100&page={list_index}'

        list = req.get(page_url, headers).json()

        if len(list) == 0:
            break
        
        followers_lists.append(list)
    except:
        print('Error while loading followers')
        followers_lists.append(None)

followers_lists


[[{'login': 'tkersey',
   'id': 217,
   'node_id': 'MDQ6VXNlcjIxNw==',
   'avatar_url': 'https://avatars.githubusercontent.com/u/217?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/tkersey',
   'html_url': 'https://github.com/tkersey',
   'followers_url': 'https://api.github.com/users/tkersey/followers',
   'following_url': 'https://api.github.com/users/tkersey/following{/other_user}',
   'gists_url': 'https://api.github.com/users/tkersey/gists{/gist_id}',
   'starred_url': 'https://api.github.com/users/tkersey/starred{/owner}{/repo}',
   'subscriptions_url': 'https://api.github.com/users/tkersey/subscriptions',
   'organizations_url': 'https://api.github.com/users/tkersey/orgs',
   'repos_url': 'https://api.github.com/users/tkersey/repos',
   'events_url': 'https://api.github.com/users/tkersey/events{/privacy}',
   'received_events_url': 'https://api.github.com/users/tkersey/received_events',
   'type': 'User',
   'site_admin': False},
  {'login': 'njonsson',
   'i